In [1]:
## Importing required Libraries
import os
import tensorflow as tf
from tensorboard.plugins import projector
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

tf.compat.v1.enable_eager_execution()

In [2]:
data = pd.read_csv('2019-Dec.csv')

In [3]:
data.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-12-01 00:00:00 UTC,remove_from_cart,5712790,1487580005268456287,NaN,f.o.x,6.27,576802932,51d85cb0-897f-48d2-918b-ad63965c12dc
1,2019-12-01 00:00:00 UTC,view,5764655,1487580005411062629,NaN,cnd,29.05,412120092,8adff31e-2051-4894-9758-224bfa8aec18
2,2019-12-01 00:00:02 UTC,cart,4958,1487580009471148064,NaN,runail,1.19,494077766,c99a50e8-2fac-4c4d-89ec-41c05f114554
3,2019-12-01 00:00:05 UTC,view,5848413,1487580007675986893,NaN,freedecor,0.79,348405118,722ffea5-73c0-4924-8e8f-371ff8031af4
4,2019-12-01 00:00:07 UTC,view,5824148,1487580005511725929,NaN,NaN,5.56,576005683,28172809-7e4a-45ce-bab0-5efa90117cd5


In [4]:
data['event_time'] = pd.to_datetime(data['event_time'],infer_datetime_format=True)

In [5]:
df = data.drop(['event_time'], axis = 1)

In [6]:
## Get working directory
PATH = os.getcwd()
## Path to save the embedding and checkpoints generated
LOG_DIR = PATH + '/project-tensorboard/log-1/'

In [7]:
# one hot encoding
event_type = pd.get_dummies(df['event_type'])
df = pd.concat([df,event_type], axis=1)


In [8]:
df = df.drop(['event_type'], axis = 1)

In [9]:
df = df.drop(['brand'], axis = 1)

In [10]:
df = df.drop(['category_code'], axis = 1)

In [11]:
df = df.drop(['user_session'], axis = 1)

In [12]:
df

,product_id,category_id,price,user_id,cart,purchase,remove_from_cart,view
0,5712790,1487580005268456287,6.27,576802932,0,0,1,0
1,5764655,1487580005411062629,29.05,412120092,0,0,0,1
2,4958,1487580009471148064,1.19,494077766,1,0,0,0
3,5848413,1487580007675986893,0.79,348405118,0,0,0,1
4,5824148,1487580005511725929,5.56,576005683,0,0,0,1
...,...,...,...,...,...,...,...,...
3533281,5683350,1487580005671109489,2.84,536812729,0,0,0,1
3533282,5888097,1487580013388628160,179.05,503658154,0,0,0,1
3533283,59975,1487580012096782476,7.14,595414541,0,0,0,1
3533284,5775982,1783999063314661546,11.90,397780878,0,0,0,1


In [13]:
# Generating PCA and 
pca = PCA(n_components=8,
         random_state = 123,
         svd_solver = 'auto'
         )
df_pca = pd.DataFrame(pca.fit_transform(df))
df_pca = df_pca.values

# Tensorflow 2 SKIP THIS PART

In [14]:
## TensorFlow Variable from data
#embeddings = tf.Variable(df_pca, name = 'embeddings')

In [20]:
'''embeddings = tf.Variable(df_pca, name='embeddings')
CHECKPOINT_FILE = LOG_DIR + '/model.ckpt'
ckpt = tf.train.Checkpoint(embeddings=embeddings)
ckpt.save(CHECKPOINT_FILE)'''

'/Users/arielliwenxi/Downloads/426888_1015349_bundle_archive/project-tensorboard/log-1//model.ckpt-1'

In [21]:
'''reader = tf.train.load_checkpoint(LOG_DIR)
map = reader.get_variable_to_shape_map()
key_to_use = ""
for key in map:
    if "embeddings" in key:
        key_to_use = key'''

In [22]:
'''config = projector.ProjectorConfig()
embedding = config.embeddings.add()
embedding.tensor_name = key_to_use'''

In [29]:
'''writer = tf.summary.create_file_writer(LOG_DIR)
projector.visualize_embeddings(writer, config)'''

TypeError: expected str, bytes or os.PathLike object, not ResourceSummaryWriter

# Tensorflow 1

In [30]:
tf.compat.v1.disable_eager_execution()

In [33]:
embeddings = tf.compat.v1.Variable(df_pca, name='embeddings')
CHECKPOINT_FILE = LOG_DIR + '/model.ckpt'
# Write summaries for tensorboard
with tf.compat.v1.Session() as sess:
    saver = tf.compat.v1.train.Saver([embeddings])
    sess.run(embeddings.initializer)
    saver.save(sess, CHECKPOINT_FILE)
    config = projector.ProjectorConfig()
    embedding = config.embeddings.add()
    embedding.tensor_name = embeddings.name
    #embedding.metadata_path = TENSORBOARD_METADATA_FILE



In [34]:
projector.visualize_embeddings(tf.compat.v1.summary.FileWriter(LOG_DIR), config)

In [37]:
CHECKPOINT_FILE

'/Users/arielliwenxi/Downloads/426888_1015349_bundle_archive/project-tensorboard/log-1//model.ckpt'

# Open Tensorboard

In [38]:
#Please run the following command in the terminal to get the tensorboard
tensorboard --logdir=...\project-tensorboard/log-1 --port=6006

SyntaxError: unexpected character after line continuation character (<ipython-input-38-f594adf2d886>, line 2)